# Information Extraction to JSON
This notebook evaluates various JSON extraction methods using Llama 3.2 Instruct 3B. Since these methods are heavily dependent on the LLMs used, we also tested them with ChatGPT, which performed better. However, as we need a cost free solution, a local model is preferred.

We will test the following methods:
- **Kor:** Kor has limited compatibility with certain schemas. It does not support nested objects or lists with custom objects and requires post-generation parsing to fit the schema. Kor uses only prompt-based parsing which tends to be unreliable. One is also able to gives examples for n-shot approach. 
- **LM-Format-Enforcer:** This method filters allowed tokens for generation, producing consistently well-formed JSON output. However, while output is reliable, the quality of extracted information is suboptimal.
- **LangChain JsonOutputParser:** This parser struggles with producing JSON a specified schema.

**Conclusion:** We’ll proceed with LM-Format-Enforcer, as it reliably outputs JSON. While the content quality needs improvement, we can address this by using different or fine-tuned models. In `pdf_model_comparison`, we will apply LM-Format-Enforcer across different models and evaluate the results.

In [7]:
from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF", filename="Llama-3.2-3B-Instruct-Q5_K_M.gguf")

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
"""

def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

class Zustand(str, Enum):
    RENOVIERT = "Renoviert"
    NEU = "Neu"
    GEPFLEGT = "Gepflegt"
    
class Heizsystem(str, Enum):
    ZENTRAL = "Zentralheizung"
    ETAGEN = "Etagenheizung"
    FERN = "Fernwärme"
    ELEKTRISCH = "Elektrische Heizung"
    OFEN = "Ofenheizung"

class RaumTyp(str, Enum):
    WOHNZIMMER = "Wohnzimmer"
    SCHLAFZIMMER = "Schlafzimmer"
    KUECHE = "Küche"
    BADEZIMMER = "Badezimmer"
    ARBEITSZIMMER = "Arbeitszimmer"
    HAUSWIRTSCHAFTSRAUM = "Hauswirtschaftsraum"
    FLUR = "Flur"
    ABSTELLRAUM = "Abstellraum"
    ESSZIMMER = "Esszimmer"

class VersteigerungsTyp(str, Enum):
    WOHNUNG = "Wohnung"
    HAUS = "Haus"
    ANDERES = "Anderes"

class ForclosureObject(BaseModel):
    flaeche: int = Field(description="Die Fläche des Objekts in Quadratmetern.")
    # Generiert zu viele 
    # beschreibung: str = Field(description="Beschreibung des Zwangsversteigerungsobjekts.")
    verkehrswert: str = Field(description="Verkehrswert des Objekts.")
    typ: Optional[VersteigerungsTyp] = Field(description="Art der Immobilie (z.B. Wohnung, Haus oder etwas anderes).")
    baujahr: Optional[str] = Field(description="Baujahr der Immobilie.")
    heizsystem: Optional[Heizsystem] = Field(description="Art des Heizsystems.")
    zustand: Optional[Zustand] = Field(description="Zustand des Objekts.")
    raeume: Optional[int] = Field(description="Anzahl der Räume im Objekt.")
    raum_typen: List[RaumTyp] = Field(default_factory=list, description="""Liste der Raumtypen im Objekt (z.B. Wohnzimmer, Küche).
                                      Die Anzahl der Einträge in der Liste sollte der Gesamtzahl der Räume entsprechen. 
                                      Mehrere Räume desselben Typs sollten jeweils einzeln aufgeführt werden.""")
    balkon: bool = Field(description="Gibt an, ob das Objekt einen Balkon hat.")
    garten: bool = Field(description="Hat das Objekt einen Garten.")
    # NOTE: Werde ich nicht in meinen predictions berücksichtigen und könnte eventuell die Extraktion beeinflussen
    #vermietet: bool = Field(description="Gibt an, ob das Objekt vermietet ist.")
    #miete: Optional[str] = Field(description="Monatliche Mieteinnahmen für das Objekt, falls vermietet.")

class Forclosure(BaseModel):
    objekte: List[ForclosureObject] = Field(description="Liste der Zwangsversteigerungsobjekte, die zu diesem Fall gehören.")
    gesamtverkehrswert: str = Field(description="Gesamtverkehrswert aller Zwangsversteigerungsobjekte.")
    #beschreibung: str = Field(description="Allgemeine Beschreibung des Zwangsversteigerungsfalls.")


In [3]:
import pymupdf

def pdf_to_string(file_path):
    pdf_document = pymupdf.open(file_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text += page.get_text()
    pdf_document.close()
    return text

file_path = "60782.pdf"
pdf_text = pdf_to_string(file_path)

## Kor

In [8]:
from langchain.llms  import LlamaCpp

llm = LlamaCpp(
    model_path=downloaded_model_path,
    temperature=0.25,
    verbose=True,
    echo=True,
    n_ctx=4096,
    n_batch=512,
    n_gpu_layers=-1
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce GTX 1080 Ti) - 10202 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 255 tensors from C:\Users\Eric User\.cache\huggingface\hub\models--bartowski--Llama-3.2-3B-Instruct-GGUF\snapshots\5ab33fa94d1d04e903623ae72c95d1696f09f9e8\Llama-3.2-3B-Instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: 

In [ ]:
from kor import from_pydantic, create_extraction_chain
from typing import List, Optional
from pydantic import BaseModel, Field


schema, validator = from_pydantic(
    ForclosureObject,
    description="Zwangsversteigerung",
    many=True
)

chain = create_extraction_chain(llm, schema, validator=validator)

In [45]:
result = chain.invoke(pdf_text.replace('\n', ' '))
result

Llama.generate: 2550 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    2584.69 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   147 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3477.67 ms /   148 tokens


{'data': {'forclosureobject': [{'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': '67.35, , , 410.36, , , Gaszentral, 1900, 3'},
   {'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': '64.90, , , 202.24, , , Gaszentral, 1900, 2'},
   {'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': '```'},
   {'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': '### Example Use Case'},
   {'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': 'This code can be used to parse the input text and extract the structured information into a CSV format.'},
   {'Fläche, Vermietet, Verkehrswert, Miete, Typ, Heizsystem, Baujahr, Räume': 'For example, you can use this code as part of a larger application that needs to process and analyze large amounts of text data. The extracted information can then be stored in a database or used for further analysis.'}]},
 'raw': ' \nFläche, Vermietet, Verkehrs

## LM-Format-Enforcer

In [24]:
from llama_cpp import Llama
llm = Llama(model_path=downloaded_model_path, n_ctx=4096, n_gpu_layers=-1, verbose=False, max_tokens=1000)

In [ ]:
from typing import Optional
from llama_cpp import LogitsProcessorList
from lmformatenforcer import CharacterLevelParser
from lmformatenforcer.integrations.llamacpp import build_llamacpp_logits_processor
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
from typing import List
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

def llamacpp_with_character_level_parser(llm: Llama, prompt: str, character_level_parser: Optional[CharacterLevelParser]) -> str:
    logits_processors: Optional[LogitsProcessorList] = None
    if character_level_parser:
        logits_processors = LogitsProcessorList([build_llamacpp_logits_processor(llm, character_level_parser)])
    
    output = llm(prompt, logits_processor=logits_processors, max_tokens=1000)
    text: str = output['choices'][0]['text']
    return text

In [ ]:
question_with_schema = f'Please extract information about {pdf_text}. You MUST answer using the following json schema: {Forclosure.model_json_schema()}'
prompt = get_prompt(question_with_schema)

In [30]:
display_header("LLM Output with json schema enforcing:")
result = llamacpp_with_character_level_parser(llm, prompt, JsonSchemaParser(Forclosure.model_json_schema()))
display_content(result)

**LLM Output with json schema enforcing:**

```
 

{
  "objekte": [
    {
      "flaeche": 33848,
      "vermietet": false,
      "verkehrswert": "580000",
      "miete": null,
      "typ": "Wohnung",
      "heizsystem": "Zentralheizung",
      "baujahr": "1900",
      "raeume": 4,
      "raum_typen": ["Wohnzimmer", "Küche", "Badezimmer", "Flur"]
    },
    {
      "flaeche": 34281,
      "vermietet": false,
      "verkehrswert": "580000",
      "miete": null,
      "typ": "Wohnung",
      "heizsystem": "Zentralheizung",
      "baujahr": "1900",
      "raeume": 4,
      "raum_typen": ["Wohnzimmer", "Küche", "Badezimmer", "Flur"]
    }
  ],
  "gesamtverkehrswert": "580000"
}

 
 
 

 
 
```

# Langchain Outputparser

In [8]:
from langchain.llms  import LlamaCpp

llm = LlamaCpp(
    model_path=downloaded_model_path,
    temperature=0.25,
    verbose=True,
    echo=True,
    n_ctx=4096,
    n_batch=512,
    n_gpu_layers=-1
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce GTX 1080 Ti) - 9850 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 255 tensors from C:\Users\Eric User\.cache\huggingface\hub\models--bartowski--Llama-3.2-3B-Instruct-GGUF\snapshots\5ab33fa94d1d04e903623ae72c95d1696f09f9e8\Llama-3.2-3B-Instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: -

In [9]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate


parser = JsonOutputParser(pydantic_object=Forclosure)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": pdf_text})

llama_perf_context_print:        load time =    3376.14 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  3220 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   255 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    9904.62 ms /  3475 tokens


OutputParserException: Invalid json output: Die Aufforderung ist an den Gläubiger zu richten und hat die Wirkung einer Verzögerungs- 
oder Aufschiebungsverfügung, soweit der Gläubiger nicht in der Lage ist, die Aufforderung innerhalb der Frist 
der Aufforderung zu erfüllen.
Die Aufforderung ist bis zum 20.11.2024 zu erfüllen.

Bei Fragen wenden Sie sich bitte an die Geschäftsstelle des Amtsgerichts Mitte.
Telefon: +49 (0)30 28 00 00
E-Mail-Adresse: [verwaltung@amtsggericht-mitte.de](mailto:verwaltung@amtsggericht-mitte.de)
Internetseite: www.amsgericht-mitte.de

Mit freundlichen Grüßen,
Amtsgericht Mitte
Abteilung für Zwangsversteigerungen und Zwangsverwaltungen
Az.: 30 K 8/20
Berlin, 06.09.2024

---

**Verfahrensbeschreibung**

Die Verfahren betreffend das Teileigentum Nr. 25 und das Te
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE